# Music Video Synthesis
* Extract lyrics from song with timestamps
* Compose scenes, include timestamps
* Construct video text prompt for each scene
* Build videos for each scene
* Stitch together

# We will use openai whipser for stability

In [ ]:
#!pip install --quiet --upgrade pip
#!pip3 install torch torchvision torchaudio torchao xformers --index-url https://download.pytorch.org/whl/cu128 -U
#!pip install --quiet --upgrade openai-whisper openai optimum-quanto 
# Ubuntu or Debian
#!sudo apt update && sudo apt install ffmpeg
#!pip install setuptools-rust -U
#!pip install -U diffusers imageio imageio_ffmpeg opencv-python moviepy transformers huggingface-hub optimum pillow safetensors ftfy -U
#!pip install -U sentencepiece einops beautifulsoup4 -U
#!pip install git+https://github.com/xhinker/sd_embed.git@main -U
#!pip install accelerate flash_attention numba -U
#!pip install flash_attn --no-build-isolation -U
#!pip install -r requirements.txt -U
#!pip install numpy==1.26.4
#!pip install peft==0.13.2

In [ ]:
import cv2
import diffusers
import gc
import imageio
import imageio_ffmpeg
import json
import math
import numpy as np
import os
import random
import tempfile
import time
import transformers
import torch
import torch.multiprocessing as mp
import whisper

from contextlib import contextmanager
from datetime import datetime, timedelta
from diffusers import AutoencoderKL, FlowMatchEulerDiscreteScheduler, LTXImageToVideoPipeline, LTXPipeline
from diffusers.pipelines.flux.pipeline_flux import FluxPipeline
from diffusers.utils import export_to_video, load_video, load_image
from huggingface_hub import hf_hub_download, snapshot_download
from model4 import T5EncoderModel as m_T5EncoderModel, FluxTransformer2DModel
from numba import cuda
from openai import OpenAI
from optimum.quanto import freeze, qfloat8, quantize, requantize
from pathlib import Path
from PIL import Image
from pipeline_stg_ltx_image2video import LTXImageToVideoSTGPipeline
from safetensors.torch import load_file as load_safetensors, save_file as save_safetensors
from sd_embed.embedding_funcs import get_weighted_text_embeddings_flux1
from torchao.quantization import quantize_, int8_weight_only, int8_dynamic_activation_int8_weight
from transformers import CLIPTextModel, CLIPTokenizer, T5TokenizerFast, T5EncoderModel

os.environ["TOKENIZERS_PARALLELISM"] = "false"
# Define the paths where quantized weights will be saved

dtype = torch.bfloat16
MAX_SEED = np.iinfo(np.int32).max
device = "cuda" if torch.cuda.is_available() else "cpu"
retry_limit = 3
quantization = int8_weight_only

WIDTH=768
HEIGHT=512
FRAME_RATE = 25
NUM_FRAMES = 151
NUM_INFERENCE_STEPS=70
GUIDANCE_SCALE=4.0

In [ ]:
# Configuration
THEME = "music video animation, erotic, beautiful, sensual, creative, imaginative, people angry about loss of freedom due to government control."
CONFIG = {
    "openai_api_key": "",
    "openai_model": "gpt-4o-mini",
    "openai_model_small_reasoning": "o1-mini",
    "openai_model_large": "gpt-4o",
    "hf_token": "",
    "base_working_dir": "./images",
    "base_video_dir": "./output",
    "audio_files": [
        "/mnt/d/audio/Pawns No More.flac",
        "/mnt/d/audio/Pawns No More.flac",
    ],
    "device": device,
    "dtype": dtype,
    "retry_limit": retry_limit,
    "MAX_SEED": MAX_SEED,
}

# Ensure base directories exist
os.makedirs(CONFIG["base_working_dir"], exist_ok=True)
os.makedirs(CONFIG["base_video_dir"], exist_ok=True)

ACTION_SEQUENCES = f'''The camera pans over a snow-covered mountain range, revealing a vast expanse of snow-capped peaks and valleys.The mountains are covered in a thick layer of snow, with some areas appearing almost white while others have a slightly darker, almost grayish hue. The peaks are jagged and irregular, with some rising sharply into the sky while others are more rounded. The valleys are deep and narrow, with steep slopes that are also covered in snow. The trees in the foreground are mostly bare, with only a few leaves remaining on their branches. The sky is overcast, with thick clouds obscuring the sun. The overall impression is one of peace and tranquility, with the snow-covered mountains standing as a testament to the power and beauty of nature.,
A woman with blood on her face and a white tank top looks down and to her right, then back up as she speaks. She has dark hair pulled back, light skin, and her face and chest are covered in blood. The camera angle is a close-up, focused on the woman's face and upper torso. The lighting is dim and blue-toned, creating a somber and intense atmosphere. The scene appears to be from a movie or TV show.
The waves crash against the jagged rocks of the shoreline, sending spray high into the air.The rocks are a dark gray color, with sharp edges and deep crevices. The water is a clear blue-green, with white foam where the waves break against the rocks. The sky is a light gray, with a few white clouds dotting the horizon.
A woman with short brown hair, wearing a maroon sleeveless top and a silver necklace, walks through a room while talking, then a woman with pink hair and a white shirt appears in the doorway and yells. The first woman walks from left to right, her expression serious; she has light skin and her eyebrows are slightly furrowed. The second woman stands in the doorway, her mouth open in a yell; she has light skin and her eyes are wide. The room is dimly lit, with a bookshelf visible in the background. The camera follows the first woman as she walks, then cuts to a close-up of the second woman's face. The scene is captured in real-life footage.
A clear, turquoise river flows through a rocky canyon, cascading over a small waterfall and forming a pool of water at the bottom.The river is the main focus of the scene, with its clear water reflecting the surrounding trees and rocks. The canyon walls are steep and rocky, with some vegetation growing on them. The trees are mostly pine trees, with their green needles contrasting with the brown and gray rocks. The overall tone of the scene is one of peace and tranquility.
A prison guard unlocks and opens a cell door to reveal a young man sitting at a table with a woman. The guard, wearing a dark blue uniform with a badge on his left chest, unlocks the cell door with a key held in his right hand and pulls it open; he has short brown hair, light skin, and a neutral expression. The young man, wearing a black and white striped shirt, sits at a table covered with a white tablecloth, facing the woman; he has short brown hair, light skin, and a neutral expression. The woman, wearing a dark blue shirt, sits opposite the young man, her face turned towards him; she has short blonde hair and light skin. The camera remains stationary, capturing the scene from a medium distance, positioned slightly to the right of the guard. The room is dimly lit, with a single light fixture illuminating the table and the two figures. The walls are made of large, grey concrete blocks, and a metal door is visible in the background. The scene is captured in real-life footage.
A woman walks away from a white Jeep parked on a city street at night, then ascends a staircase and knocks on a door. The woman, wearing a dark jacket and jeans, walks away from the Jeep parked on the left side of the street, her back to the camera; she walks at a steady pace, her arms swinging slightly by her sides; the street is dimly lit, with streetlights casting pools of light on the wet pavement; a man in a dark jacket and jeans walks past the Jeep in the opposite direction; the camera follows the woman from behind as she walks up a set of stairs towards a building with a green door; she reaches the top of the stairs and turns left, continuing to walk towards the building; she reaches the door and knocks on it with her right hand; the camera remains stationary, focused on the doorway; the scene is captured in real-life footage.
'''

SCENE_DESCRIPTIONS = '''scene_description: boris vallejo style, frank frazetta style, 8k high quality digital painting, masterpiece, very detailed, ultra realistic, (best quality) very detailed epic masterpiece, detailed face, full body, wrinkly wizard toad reading an ancient scroll in a swamp, best quality, epic scene, Dungeons and dragons atmosphere, heroic fantasy, realistic, realism, full body
scene_description: masterpiece, best quality, amazing quality ,solo, holding, closed_mouth, sitting, outdoors, sky, day, cloud, water, blurry, blue_sky, tree, orange_eyes, no_humans, blurry_background, fish, reflection, mountain, animal_focus, lake, fishing_rod, reflective_water, fishing, holding_fishing_rod, fishing_line,A digital illustration shoot from the side about a cute cartoon fish character sitting on a wooden pier by a calm lake, holding a fishing rod. the image also shows a serene mountain landscape with tall trees and a clear blue sky. on the middle of the image, a no human, furry, blue and orange fish with large, expressive eyes and a happy expression is sitting on the wooden pier. the fish appears to be a chubby, cartoonish creature with a slim body and a closed mouth. it is facing the viewer with its eyes looking to the side. the creature is holding the fishing rod in its right hand and its left hand is resting on the edge of the water. the background features a mountain range with a few clouds in the sky, and the water is calm and still. the lighting is soft and natural, creating a peaceful and serene atmosphere. solo, looking at viewer, closed mouth, sitting, brown eyes, outdoors, sky, day, cloud, holding, tree, blue sky, water, tree branch, holding stick, mountain, fish, pond, holding fishing rod
scene_description: masterpiece, best quality, good quality, very awa, newest, highres, absurdres, 1girl, solo, dress, standing, flower, outdoors, water, white flower, pink flower, scenery, reflection, rain, dark, ripples, yellow flower, puddle, colorful, abstract, standing on liquid, very Wide Shot, limited palette,
scene_description: masterpiece, best quality, amazing quality, klskx, nsfw, explicit, 1girl, redhead, open fridge, dim blue fridge light, nude, oversized t-shirt slipping off shoulder, panties, barefoot, messy hair, licking fingers, one hand on hip, standing by fridge, looking back, midnight snack indoors, kitchen, wooden floor, open fridge door, scattered snacks, napkin, night, dark shadows, high contrast, volumetric lighting, intricate details, blurry background, depth of field
scene_description: time travel, holding coffee,, hdr, 8k, absurdres, shiny, outdoors, reflection, blurry, blurry background, tokyo lights,tokyo street, neon lights, cyberpunk, high-contrast lighting, intricate details, vibrant colors, reflective surfaces, futuristic urban environment, glowing neon signs, cybernetic enhancements, punk aesthetic, dynamic pose, dynamic composition, depth of field, dark_theme, detailed backgroud, foreshortening, blurry edges, vignetting
scene_description: masterpiece, best quality, amazing quality ,solo, yellow_eyes, flower, outdoors, sky, cloud, tree, no_humans, night, animal, facial_mark, moon, cat, star_\(sky\), night_sky, full_moon, starry_sky, animal_focus, architecture, black_cat, east_asian_architecture, whiskers, black_fur, huge_moon,A digital illustration shoot from a front camera angle about a cute black cat sitting on a moss-covered branch under a large full moon, surrounded by a fantasy setting with tall pagodas and flowers. on the middle of the image, a 1girl, who appears to be a cat, is sitting, looking at the viewer with large, expressive brown eyes. the cat has black fur with yellow stripes, and its ears are perked up, giving it a curious expression. it is positioned on the branch, with its body facing the viewer, giving a clear view of its full body. in the background, a full moon is visible, with stars twinkling in the night sky, and fluffy white clouds are visible, adding to the magical atmosphere of the scene. the overall style is whimsical and fantastical, with a focus on the cat's curious expression and the intricate details of its markings on its body. solo, smile, brown eyes, flower, outdoors, sky, day, cloud, no humans, animal, night, animal focus, star \(sky\), moon, cat, full moon
scene_description: hyper realistic, a majestic A gemstone stag slowly blooming into life, moss and flowers sprouting from cracks in its crystalline body as it awakens, its eyes, initially dull stones, begin to glow with an inner emerald light, dawn light filters through a forest, illuminating the stags nascent awakening, wide shot capturing the stags full form and the blooming flora with golden antlers standing in a sunlit clearing, surrounded by ethereal forest spirits, glowing flora, magical atmosphere, extremely high-resolution details, photographic, realism pushed to extreme, fine texture, incredibly lifelike, Cinematic, beautiful, vibrant, masterpiece, 32k, ultra HD, ultra-detailed, amazing quality, amazing artist, sharp edges, detailed textures, full view, atmospheric lighting, amazing visuals
scene_description: impressive and grotesque scenery on a distant world, famous artwork inspired by jordan grimmer, dramatic scene, fractal art, 1990s fantasy style, dynamic angle, this image shows the enormous transcendent hydra-like beast known as the powerful jadesnap as it wriggleflomps from the izzled depths of a churning, otherworldly glowing sea under a dramatic tempestuous sky, it is surrounded by the typical jagged fractal rock formations on the crinkled shore of the water-rich planet "zoffeldirly quartus," superstitious life forms call it "the bringer of lost keys", creature focus, very aesthetic, extremely detailed, ultra high resolution, 8k, 4k, harmonising colors, light beige and chartreuse and bordeaux red and indigo blue and byzantium purple and ebony black, ovg, in the style of ck-ovf, amanoer, arsmjstyle, dnddarkestfantasy, aidmafluxpro1.1
scene_description: A realistic toilet, completely engulfed in flames, inferno, blazing, concept art, masterpiece, perfect lighting, purple and pink flames, realistic flames, 8k, absurdres, massive fire - rendered in the highest quality, realistic bathroom background, A3ther
scene_description: intricate linework with expressive contrasts, soft lighting with dynamic highlights, young woman wearing flight goggles, aviator leather jacket, long loose platinum hair, standing next to a 1930s biplane on an airstrip surrounded by tropical jungle, sunset, in orange hues
scene_description: 1 girl , ghost girl , grave stone , hugging , kneeling , tear , raining , masterpiece, best quality, good quality, very awa, newest, highres, absurdres
scene_description: Indi_and_Digo,1girl,solo,furry,tail,source_furry,,red hair,purple fur, purple eyes, child, kid, masterpiece, best quality, female hand holding a small umbrella, miniature wet tiny mouse standing on the path, rain, drops, funny, intricate details, hyper-realistic, hyper-detailed, professional photoshoot, colorful, ultra-sharp, vivid color, chiaroscuro lighting, macro
scene_description: masterpiece, best quality, good quality, very aesthetic, absurdres, newest, 8K, depth of field, in the style of cknc, artist:moriimee, in the style of cksc, 1girl, short golden hair, bob cut, blue eyes, large breasts, bouncy, baggy red tank top, sagging, oversized leggings, sweat-soaked, soft moan, looking up, from below, hiking on ridge, pressing breasts through tank, nipples outlined BREAK mountain ridge, rocky path, whistling wind, pine scent BREAK warm light, high contrast, earthy tones, depth of field, golden hour lighting, rich details, rugged allure
scene_description: masterpiece, best quality, good quality, very aesthetic, absurdres, newest, 8K, depth of field, in the style of cknc, artist:moriimee, in the style of cksc, 1girl, short green hair, tangled, hazel eyes, medium breasts, firm, hooded cloak, glossy skin, picking lock, smirking, looking at viewer, from side BREAK twilight forest, mossy ruins, owls hooting BREAK dynamic light, green tones, high contrast, twilight glow, rich details, sneaky atmosphere
scene_description: A digital art splash in the style of bo-cyborgsplash, a mysterious raven character positioned in the center of the frame, directly facing the viewer, the raven's upper body is close to the camera, showcasing its dark, ornate attire adorned with intricate details and glowing purple gemstones, its long beak is visible, and its piercing pink eyes seem to lock onto the viewer with a sense of intensity, the background is a dark, neon-lit space with glowing elements, creating a mystical atmosphere, the character is adorned with numerous gemstone necklaces, adding a touch of opulence to the overall design, the overall effect is one of intrigue and mystery, a fantastic abstract colorful art splash, high quality, ultra detailed
scene_description: masterpiece, best quality, good quality, very aesthetic, absurdres, newest, 8K, depth of field, in the style of cknc, artist:moriimee, in the style of cksc, 1girl, long grey hair, wild, stormy eyes, large breasts, round, thunderbolt crown, glossy skin, summoning lightning, looking at viewer, from below BREAK stormy cliff, dark clouds, waves crashing BREAK dynamic light, blue tones, high contrast, electric glow, rich details, dramatic atmosphere
scene_description: masterpiece, hyper detailed, high quality v3, (ultra-HD details), 16k, midjourneyv6.1, (Pencil_Sketch:1.2, messy lines, greyscale, traditional media, sketch), anime, manga, sketch, unfinished, hatching (texture), fullbody portrait, long legs, mustard XXDFace head, creepy smile, x-eyes, robot joints, wearing ((creepy beige victorian tattered, broken, cracked, dirt, ripped dress)) at night, in 22@SIT_BCN wide street, creepy red moonlight, perfect anatomy
scene_description: masterpiece, best quality, good quality, very awa, newest, highres, absurdres, 1girl, solo, short hair, blonde hair, red eyes, holding, standing, belt, hood, water, scarf, arm up, torn clothes, plant, hood up, wading, partially submerged, mittens, pillar, torch, holding torch, limited palette
scene_description: anthropomorphic corgi knight, corgi head, on one knee, planted sword, holding sword, plate armor, scowl, v-shaped eyebrows ,cloudy, godrays, sunshine, riverbank background, holy halo ,wide shot, depth of field, realism, no humans, animal focus, corpses,covered in blood, battlefield
scene_description: 1girl,solo,furry,pink fur,tail,ears,source_furry, child, kid, masterpiece, best quality, long hair, twin braids, farmer outfit,(steampunk), goggles, googles on head, bag, wheat field, outdoors, wind, accordion, holding instrument, playing instrument, gloves, wheat, sunset, farm, house, scenery, landscape, blurry, blurry background, looking at viewer, smile
scene_description: masterpiece, best quality, amazing quality, solo, sitting, no humans, glowing, wariza, robot, science fiction, on floor, electricity, cable, joints, robot joints, damaged, mechanical parts, wire, humanoid robot, screw, bolt, Countless lightning, electric shock, open mouth, Open five fingers,hands up, Bent back, Low Angle
'''

In [ ]:
def center_crop_and_resize(frame, target_height, target_width):
    h, w, _ = frame.shape
    aspect_ratio_target = target_width / target_height
    aspect_ratio_frame = w / h
    if aspect_ratio_frame > aspect_ratio_target:
        new_width = int(h * aspect_ratio_target)
        x_start = (w - new_width) // 2
        frame_cropped = frame[:, x_start : x_start + new_width]
    else:
        new_height = int(w / aspect_ratio_target)
        y_start = (h - new_height) // 2
        frame_cropped = frame[y_start : y_start + new_height, :]
    frame_resized = cv2.resize(frame_cropped, (target_width, target_height))
    return frame_resized


def load_video_to_tensor_with_resize(video_path, target_height, target_width):
    cap = cv2.VideoCapture(video_path)
    frames = []
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        if target_height is not None:
            frame_resized = center_crop_and_resize(
                frame_rgb, target_height, target_width
            )
        else:
            frame_resized = frame_rgb
        frames.append(frame_resized)
    cap.release()
    video_np = (np.array(frames) / 127.5) - 1.0
    video_tensor = torch.tensor(video_np).permute(3, 0, 1, 2).float()
    return video_tensor


def load_image_to_tensor_with_resize(image_path, target_height=512, target_width=768):
    image = Image.open(image_path).convert("RGB")
    image_np = np.array(image)
    frame_resized = center_crop_and_resize(image_np, target_height, target_width)
    frame_tensor = torch.tensor(frame_resized).permute(2, 0, 1).float()
    frame_tensor = (frame_tensor / 127.5) - 1.0
    # Create 5D tensor: (batch_size=1, channels=3, num_frames=1, height, width)
    return frame_tensor.unsqueeze(0).unsqueeze(2)
    
def reset_memory(device):
    gc.collect()
    torch.cuda.empty_cache()
    torch.cuda.reset_peak_memory_stats(device)
    torch.cuda.reset_accumulated_memory_stats(device)
    
def get_openai_prompt_response(
    prompt: str,
    config: dict,
    max_tokens: int = 6000,
    temperature: float = 0.33,
    openai_model: str = "",
):
    """
    Sends a prompt to OpenAI's API and retrieves the response with retry logic.
    """
    client = OpenAI(api_key=config["openai_api_key"])
    response = client.chat.completions.create(
        max_tokens=max_tokens,
        messages=[
            {
                "role": "system",
                "content": """Act as a helpful assistant, you are an expert editor.""",
            },
            {"role": "user", "content": prompt},
        ],
        model=openai_model,
        temperature=temperature,
    )

    retry_count = 0
    while retry_count < config["retry_limit"]:
        try:
            message_content = response.choices[0].message.content
            return message_content
        except Exception as e:
            print(f"Error occurred: {e}")
            retry_count += 1
            if retry_count == config["retry_limit"]:
                print("Retry limit reached. Moving to the next iteration.")
                return ""
            else:
                print(f"Retrying... (Attempt {retry_count}/{config['retry_limit']})")
                time.sleep(1)  # Optional: wait before retrying

def get_openai_prompt_response_reasoning(
    prompt: str,
    config: dict,
    max_tokens: int = 6000,
    openai_model: str = "",
):
    """
    Sends a prompt to OpenAI's API and retrieves the response with retry logic.
    """
    client = OpenAI(api_key=config["openai_api_key"])
    response = client.chat.completions.create(
        max_completion_tokens=max_tokens,
        messages=[
            {"role": "user", "content": prompt},
        ],
        model=openai_model,
    )

    retry_count = 0
    while retry_count < config["retry_limit"]:
        try:
            message_content = response.choices[0].message.content
            return message_content
        except Exception as e:
            print(f"Error occurred: {e}")
            retry_count += 1
            if retry_count == config["retry_limit"]:
                print("Retry limit reached. Moving to the next iteration.")
                return ""
            else:
                print(f"Retrying... (Attempt {retry_count}/{config['retry_limit']})")
                time.sleep(1)  # Optional: wait before retrying


def load_flux_pipe():
    bfl_repo = "black-forest-labs/FLUX.1-dev"
    revision = "refs/pr/3"
    adapter_id = "alimama-creative/FLUX.1-Turbo-Alpha"

    scheduler = FlowMatchEulerDiscreteScheduler.from_pretrained(bfl_repo, subfolder="scheduler", revision=revision)
    text_encoder = CLIPTextModel.from_pretrained("openai/clip-vit-large-patch14", torch_dtype=dtype)
    tokenizer = CLIPTokenizer.from_pretrained("openai/clip-vit-large-patch14", torch_dtype=dtype)
    text_encoder_2 = T5EncoderModel.from_pretrained(bfl_repo, subfolder="text_encoder_2", torch_dtype=dtype, revision=revision)
    tokenizer_2 = T5TokenizerFast.from_pretrained(bfl_repo, subfolder="tokenizer_2", torch_dtype=dtype, revision=revision)
    vae = AutoencoderKL.from_pretrained(bfl_repo, subfolder="vae", torch_dtype=dtype, revision=revision)
    transformer = FluxTransformer2DModel.from_pretrained(bfl_repo, subfolder="transformer", torch_dtype=dtype, revision=revision)
    
    quantize_(transformer, quantization())
    quantize_(text_encoder_2, quantization())
    pipe = FluxPipeline(
        scheduler=scheduler,
        text_encoder=text_encoder,
        tokenizer=tokenizer,
        text_encoder_2=text_encoder_2,
        tokenizer_2=tokenizer_2,
        vae=vae,
        transformer=transformer,
    )

    pipe = pipe.to('cuda')
    pipe.load_lora_weights(adapter_id)

    return pipe


def gen_flux_image(pipe, prompt, config: dict, height=1080, width=1920, guidance_scale=3.5, num_inference_steps=8, max_sequence_length=512, seed=-1):
    """
    Generates an image based on the provided prompt using the Flux pipeline.
    """
    prompt = "masterpiece, " + prompt
    if seed == -1:
        seed = random.randint(0, MAX_SEED)
        
    with torch.no_grad():
        prompt_embeds, pooled_prompt_embeds = get_weighted_text_embeddings_flux1(
            pipe        = pipe,
            prompt    = prompt
        )
        
        image = pipe(
            prompt_embeds               = prompt_embeds,
            pooled_prompt_embeds      = pooled_prompt_embeds,
            height=height,
            width=width,
            guidance_scale=guidance_scale,
            output_type="pil",
            num_inference_steps=num_inference_steps,
            max_sequence_length=max_sequence_length,
            generator=torch.Generator("cpu").manual_seed(seed)
        ).images[0]

        return image


def load_video_pipeline():
    """
    Loads and configures the video generation pipeline.
    """
    #pipe = LTXImageToVideoPipeline.from_pretrained("Lightricks/LTX-Video", torch_dtype=torch.bfloat16)
    pipe = LTXImageToVideoSTGPipeline.from_pretrained("Lightricks/LTX-Video", torch_dtype=torch.bfloat16)
    pipe.to("cuda")

    return pipe

def generate_video(pipeline, prompt, image_input, config: dict, seed_value: int = -1, video_filename: str = "", num_frames: int = 151):
    """
    Generates and saves a video from the provided image and prompt.
    """
    negative_prompt = "low quality, worst quality, deformed, distorted, disfigured, inconsistent motion, motion smear, motion artifacts, fused fingers, bad anatomy, weird hand, ugly, fast motion, jittery, text, subtitles, captions, dark figures, silhouette, transition, scene change, inconsistent motion, blurry, watermarks"
    if seed_value == -1:
        seed_value = random.randint(0, 255)

    stg_mode = "STG-R" # STG-A, STG-R
    # set to 2 orig, default is 8, 14 is best
    stg_applied_layers_idx = [14] # 0~27
    stg_scale = 0.8 # 0.0 for CFG

    image = load_image(image_input)

    try:
        with torch.no_grad():
            video = pipeline(
                image=image,
                prompt=prompt,
                negative_prompt=negative_prompt,
                width=WIDTH,
                height=HEIGHT,
                num_frames=num_frames,
                num_inference_steps=NUM_INFERENCE_STEPS,
                guidance_scale=GUIDANCE_SCALE,
                generator=torch.manual_seed(seed_value),
                stg_mode=stg_mode,
                stg_applied_layers_idx=stg_applied_layers_idx,
                stg_scale=stg_scale,
        ).frames[0]
        export_to_video(video, video_filename, fps=24)
        
    except Exception as e:
        print(f"An error occurred while generating the video. Please try again. Error: {e}")
    finally:
        gc.collect()
        torch.cuda.empty_cache()

    return video_filename


def save_video(frames, fps: int, filename: str):
    """
    Saves a list of frames as a video file.
    """
    with tempfile.NamedTemporaryFile(suffix=".mp4", delete=False) as temp_file:
        temp_video_path = temp_file.name
        writer = imageio.get_writer(temp_video_path, fps=fps)
        for frame in frames:
            writer.append_data(np.array(frame))
        writer.close()

    os.rename(temp_video_path, filename)
    return filename


def convert_to_gif(video_path: str) -> str:
    """
    Converts a video file to a GIF.
    """
    clip = mp.VideoFileClip(video_path)
    clip = clip.set_fps(8)
    clip = clip.resize(height=240)
    gif_path = video_path.replace(".mp4", ".gif")
    clip.write_gif(gif_path, fps=8)
    return gif_path


def resize_if_unfit(input_video: str) -> str:
    """
    Resizes the video to the target dimensions if it does not match.
    """
    width, height = get_video_dimensions(input_video)

    if width == 720 and height == 480:
        return input_video
    else:
        return center_crop_resize(input_video)


def get_video_dimensions(input_video_path: str) -> tuple:
    """
    Retrieves the dimensions of the video.
    """
    reader = imageio_ffmpeg.read_frames(input_video_path)
    metadata = next(reader)
    return metadata["size"]


def center_crop_resize(input_video_path: str, target_width: int = 720, target_height: int = 480) -> str:
    """
    Resizes and center-crops the video to the target dimensions.
    """
    cap = cv2.VideoCapture(input_video_path)

    orig_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    orig_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    orig_fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    width_factor = target_width / orig_width
    height_factor = target_height / orig_height
    resize_factor = max(width_factor, height_factor)

    inter_width = int(orig_width * resize_factor)
    inter_height = int(orig_height * resize_factor)

    target_fps = 8
    ideal_skip = max(0, math.ceil(orig_fps / target_fps) - 1)
    skip = min(5, ideal_skip)  # Cap at 5

    while (total_frames / (skip + 1)) < 49 and skip > 0:
        skip -= 1

    processed_frames = []
    frame_count = 0
    total_read = 0

    while frame_count < 49 and total_read < total_frames:
        ret, frame = cap.read()
        if not ret:
            break

        if total_read % (skip + 1) == 0:
            resized = cv2.resize(frame, (inter_width, inter_height), interpolation=cv2.INTER_AREA)

            start_x = (inter_width - target_width) // 2
            start_y = (inter_height - target_height) // 2
            cropped = resized[start_y:start_y + target_height, start_x:start_x + target_width]

            processed_frames.append(cropped)
            frame_count += 1

        total_read += 1

    cap.release()

    with tempfile.NamedTemporaryFile(suffix=".mp4", delete=False) as temp_file:
        temp_video_path = temp_file.name
        fourcc = cv2.VideoWriter_fourcc(*"mp4v")
        out = cv2.VideoWriter(temp_video_path, fourcc, target_fps, (target_width, target_height))

        for frame in processed_frames:
            out.write(frame)

        out.release()

    return temp_video_path


def extract_last_frame(video_filename: str, output_image_filename: str):
    """
    Extracts the last frame from a video file and saves it as an image.
    """
    try:
        reader = imageio.get_reader(video_filename, 'ffmpeg')
        last_frame = None
        for frame in reader:
            last_frame = frame
        reader.close()

        if last_frame is not None:
            imageio.imwrite(output_image_filename, last_frame)
            print(f"Last frame saved successfully as '{output_image_filename}'.")
        else:
            print("The video contains no frames.")

    except FileNotFoundError:
        print(f"Error: The file '{video_filename}' was not found.")
    except ValueError as ve:
        print(f"ValueError: {ve}")
    except RuntimeError as re:
        print(f"RuntimeError: {re}")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")


def create_scenes(text: str, video_summary: str, config: dict):
    """
    Creates scenes based on the extracted lyrics using OpenAI's API.
    """
    # Generate scenes JSON
    prompt = f'''Create a json list of diverse, unique scenes (groupings of text), scene_description, and action_sequence (100 words or less) from the following text.  Scenes should be groups of lyrics with new scenes when the lyric context changes.  Text: {text}   
The json list should have the start value for the first item in the scene and the text that is combined for all items in the same scene.  
The scene_description should include sensory rich details such as attire, setting, mood, lighting, and changing compositions, painting a clear visual scene consistent with the video theme and different from other scenes.  
Use theme descriptions, such as graphic novel, water color, render, oil painting, etc.  When people are present in a scene, focus on a closeup of a single person, preferably anthropormorphic characters.
Favor symbolism and artistic illustrations of concepts and feeling, emotion.  Favor nudity, semi-nude like see through and sheer, and natural beauty, including artistic horror when relevant.
Evoke strong emotions with atmospheric lighting and tone. Use descriptive phrases to capture the mood (e.g., flickering neon, golden twilight, ethereal glow). Use religious references and analogies.
Use special effects and CGI in scene descriptions, like glowing eyes, magical environments, electric auras, spiritual energy, etc. Use lots of CGI, special effects for a magical experience.
The scene description should include spiritual aspects, symbolism, and artistic concepts with genius expression. Avoid scenes with more than a single person in a closeup.
The action_sequence should describe the action in the scene.  Scenes should be unique, creative, imaginative, and awe-inspiring to create an amazing video.  Create beautiful and mesmerizing scene descriptions that are creative, unique, artistic, and imaginative. Each scene must be unique, imaginative, and visually captivating, blending creativity with artistic flair. Use powerful, descriptive language to craft scenes that are awe-inspiring and leave the audience in wonder. These scenes should evoke a sense of beauty, grandeur, mystery, or anything emotional, drawing from both realistic and fantastical elements. Ensure the descriptions are immersive, emotionally resonant, and filled with unexpected twists that engage the senses and imagination, suitable for creating a stunning, cinematic video experience.  Use descriptions of special effects in the scenes.  
Action should avoid sudden or fast movement or zooms, avoid any fast camera movement.  Avoid human movements like walking, dancing, shopping, etc.  Avoid scene transitions.
Make sure the scene desriptions and action sequences result in an artistic cinematic masterpiece full of inspiration, wonder, creativity, and imagination.
The overall theme is: {THEME}
These are example scene_descriptions: {SCENE_DESCRIPTIONS}
Here are some examples of action_sequences: {ACTION_SEQUENCES}
Return only the json list, less jargon. The json list fields should be: start, text, scene_description, action_sequence'''

    result = get_openai_prompt_response(prompt, config, openai_model=config["openai_model"], temperature=0.66)
    result = result.replace("```", "").replace("```json\n", "").replace("json\n", "").replace("\n", "")
    scenes = json.loads(result)
    return scenes

def revise_scenes(scenes, config: dict):
    """
    Revise scenes based on the extracted scenes.
    """
    # Generate scenes JSON
    prompt = f'''Revise the JSON scenes to update the scene_description and action_sequence to engage the senses and imagination, suitable for creating a stunning, cinematic video experience.  We want unique scenes, even ones in the same sequence. Use descriptions of special effects in the scenes.  JSON scenes: {scenes}   
The scene_description should include sensory rich details such as attire, setting, mood, lighting, and changing compositions, painting a clear visual scene consistent with the video theme and different from other scenes.  
Use theme descriptions, such as graphic novel, water color, render, oil painting, etc.  When people are present in a scene, focus on a closeup of a single person, preferably anthropormorphic characters.
Favor symbolism and artistic illustrations of concepts and feeling, emotion.  Favor nudity, semi-nude like see through and sheer, and natural beauty, including artistic horror when relevant.
Evoke strong emotions with atmospheric lighting and tone. Use descriptive phrases to capture the mood (e.g., flickering neon, golden twilight, ethereal glow). Use religious references and analogies.
Use special effects and CGI in scene descriptions, like glowing eyes, magical environments, electric auras, spiritual energy, etc. Use lots of CGI, special effects for a magical experience.
The scene description should include spiritual aspects, symbolism, and artistic concepts with genius expression. Avoid scenes with more than a single person in a closeup.
The action_sequence should describe the action in the scene.  Scenes should be unique, creative, imaginative, and awe-inspiring to create an amazing video.  Create beautiful and mesmerizing scene descriptions that are creative, unique, artistic, and imaginative. Each scene must be unique, imaginative, and visually captivating, blending creativity with artistic flair. Use powerful, descriptive language to craft scenes that are awe-inspiring and leave the audience in wonder. These scenes should evoke a sense of beauty, grandeur, mystery, or anything emotional, drawing from both realistic and fantastical elements. Ensure the descriptions are immersive, emotionally resonant, and filled with unexpected twists that engage the senses and imagination, suitable for creating a stunning, cinematic video experience.  Use descriptions of special effects in the scenes.  
Action should avoid sudden or fast movement or zooms, avoid any fast camera movement.  Avoid human movements like walking, dancing, shopping, etc.  Avoid scene transitions.
Make sure the scene desriptions and action sequences result in an artistic cinematic masterpiece full of inspiration, wonder, creativity, and imagination.
The overall theme is: {THEME}
These are example scene_descriptions: {SCENE_DESCRIPTIONS}
Here are some examples of action_sequences: {ACTION_SEQUENCES}
The action_sequence (100 words or less) should describe the action in the scene.  The goal is to create input to create a stunning, cinematic video experience.   
Action should avoid sudden or fast movements or zooms, avoid any fast movements, avoid any fast camera movements. 
Action sequences should include camera instructions.
Only update the scene_description and action_sequence. We do not want to have similar scene_descriptions and action_sequences for consecutive scenes, we want unique scenes that tell a brilliant, cohesive story.  Please update the scene_description and action_sequence to be different, creative, and consistent.  
Do not delete any items as having scenes with the given start times are important. 
Return only the json list, less jargon. The json list fields should be: start, text, scene_description, action_sequence'''

    result = get_openai_prompt_response_reasoning(prompt, config, openai_model=config["openai_model_small_reasoning"])
    result = result.replace("```", "").replace("```json\n", "").replace("json\n", "").replace("\n", "")
    scenes = json.loads(result)
    return scenes


def process_audio_scenes(audio_file: str, config: dict):
    # set maximum duration for an image basis, should be in intervals of video generation length
    video_gen_length = 6
    max_duration_seconds  = video_gen_length * 2
    """
    Processes a single audio file through the entire workflow.
    """
    # Create unique identifier based on audio file name
    audio_basename = os.path.splitext(os.path.basename(audio_file))[0]
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    unique_id = f"{audio_basename}_{timestamp}"

    # Create unique directories for images and videos
    print(f"Create unique directories for images and videos")
    audio_images_dir = os.path.join(config["base_working_dir"], unique_id)
    audio_videos_dir = os.path.join(config["base_video_dir"], unique_id)
    os.makedirs(audio_images_dir, exist_ok=True)
    os.makedirs(audio_videos_dir, exist_ok=True)

    # Step 1: Transcribe audio using Whisper
    print(f"Transcribe audio using Whisper")
    model = whisper.load_model("turbo")
    result = model.transcribe(audio_file)

    # Cleanup Whisper model memory
    del model
    reset_memory(device)

    segments = result['segments']

    # Extract list of start times and texts
    segment_texts_and_start_times = [(segment['text'].strip(), segment['start']) for segment in segments]

    # Combine texts
    text = ""
    for segment_text, start in segment_texts_and_start_times:
        text += f"Start: {start}, Text: {segment_text}\n"

    last_end_value = segments[-1]['end']

    # Path to scenes.json file
    scenes_file_path = os.path.join(audio_images_dir, "scenes.json")

    # Check if scenes.json exists
    if os.path.exists(scenes_file_path):
        print(f"Scenes file already exists at {scenes_file_path}. Skipping scene generation.")
        with open(scenes_file_path, "r") as scenes_file:
            scenes = json.load(scenes_file)
        return scenes, audio_images_dir, audio_videos_dir, last_end_value

    # Step 2: Generate video summary using OpenAI
    print(f"Generate video summary using OpenAI")
    video_summary_prompt = f'Create a short summary that describes a music video based on these lyrics: {text}'
    video_summary = get_openai_prompt_response(video_summary_prompt, config, openai_model=config["openai_model"])

    # Step 3: Create scenes based on lyrics
    print(f"Create scenes based on lyrics")
    try:
        scenes = create_scenes(text, video_summary, config)
    except:
        try:
            scenes = create_scenes(text, video_summary, config)
        except:
            try:
                scenes = create_scenes(text, video_summary, config)
            except: 
                return "", audio_images_dir, audio_videos_dir, last_end_value
            
    # we don't want scenes longer than 18 seconds
    new_scenes = []
    for i in range(len(scenes)):
        scene = scenes[i]
        if i == 0:
            start_time = 0
        else:
            start_time = scene['start']
        # Determine the end time
        if i < len(scenes) - 1:
            end_time = scenes[i + 1]['start']
        else:
            end_time = last_end_value
        duration = end_time - start_time
        # Split the scene if duration exceeds 18 seconds
        while duration > 18:
            new_scene = scene.copy()
            new_scene['start'] = start_time
            new_scenes.append(new_scene)
            start_time += max_duration_seconds
            duration = end_time - start_time
        # Append the remaining part of the scene
        if duration > 0:
            new_scene = scene.copy()
            new_scene['start'] = start_time
            new_scenes.append(new_scene)
    # Replace the original scenes with the new list
    scenes = new_scenes
    # improve the scenes with a revision
    try:
        scenes_revised = revise_scenes(scenes, config)
        scenes = scenes_revised
        print(f'revised scenes')
    except:
        try:
            scenes_revised = revise_scenes(scenes, config)
            scenes = scenes_revised
            print(f'revised scenes')
        except:
            print('cannot revise scenes')
            
    
    # Save the scenes to scenes.json
    with open(scenes_file_path, "w") as scenes_file:
        json.dump(scenes, scenes_file)
        
    return scenes, audio_images_dir, audio_videos_dir, last_end_value, timestamp

def process_audio_images(config: dict, scenes, audio_images_dir):
    # Step 4: Load Flux pipeline and generate images
    print(f"Load Flux pipeline and generate images")
    flux_pipe = load_flux_pipe()
    height = HEIGHT
    width = WIDTH
    guidance_scale = 3.9
    num_inference_steps = 16
    max_sequence_length = 512
    seed = -1

    # Generate images for each scene
    image_num = 1
    for scene in scenes:
        image_prompt = scene['scene_description']
        image = gen_flux_image(flux_pipe, image_prompt, config, height, width, guidance_scale, num_inference_steps, max_sequence_length, seed)
        filename = f"image_{str(image_num).zfill(2)}.jpg"
        image_path = os.path.join(audio_images_dir, filename)
        image.save(image_path, dpi=(300, 300))
        image_num += 1

    # Move the pipeline back to CPU and delete it
    flux_pipe.to('cpu')
    del flux_pipe
    reset_memory(device)
    return

def process_audio_video(config: dict, scenes, audio_images_dir, audio_videos_dir, last_end_value, timestamp):
    # Step 6: Load Video Pipeline
    print(f"Load Video Pipeline")
    video_pipe = load_video_pipeline()

    # Temporary image path
    temp_image = os.path.join(audio_images_dir, "temp_image.jpg")
    video_num = 1

    # Step 7: Generate video sequences
    for i, scene in enumerate(scenes):
        prompt = scene["action_sequence"]

        # Use the initial image for each scene
        image_input = os.path.join(audio_images_dir, f"image_{str(i+1).zfill(2)}.jpg")

        # Calculate duration to keep the video in 6-second increments
        if i + 1 < len(scenes):
            next_start_time = scenes[i + 1]["start"]
        else:
            next_start_time = last_end_value  # Use the final ending time for the last scene

        if i == 0:
            duration = next_start_time
        else:
            duration = next_start_time - scene["start"]
        num_video_segments = int((duration + 3) // 6)

        print(f"Scene {i+1} has {num_video_segments} segments")
        for j in range(num_video_segments):
            video_name = f"v_ltx_{str(video_num).zfill(2)}_{str(i+1)}_{str(j+1).zfill(2)}_{timestamp}.mp4"
            video_output_path = os.path.join(audio_videos_dir, video_name)
            generate_video(video_pipe, prompt, image_input, config, seed_value=-1, video_filename=video_output_path)
            time.sleep(1)  # Pause for 1 second

            # After generating the video, extract the last frame to use as input for the next segment
            extract_last_frame(video_output_path, temp_image)

            # Use the last frame as input for the next video segment in the same scene
            image_input = temp_image

            video_num += 1  # Increment video number for the next segment

    # Move the pipeline back to CPU before deleting
    video_pipe.to('cpu')
    del video_pipe
    reset_memory(device)
    
    return


def process_all_audios(audio_file, config: dict):
    """
    Processes a list of audio files through the workflow.
    """
    print(f"Processing audio file: {audio_file}")
    scenes, audio_images_dir, audio_videos_dir, last_end_value, timestamp = process_audio_scenes(audio_file, config)
    print(f'{len(scenes)} scenes:\n{json.dumps(scenes, indent=4)}')
    print(f'last_end_value: {last_end_value} timestamp: {timestamp}')
    # Create starting images for scenes
    process_audio_images(config, scenes, audio_images_dir)
    return config, scenes, audio_images_dir, audio_videos_dir, last_end_value, timestamp

def create_video():
    config, scenes, audio_images_dir, audio_videos_dir, last_end_value, timestamp = process_all_audios(audio_file, CONFIG)
    process_audio_video(config, scenes, audio_images_dir, audio_videos_dir, last_end_value, timestamp)
    return
    


### Run new

In [ ]:
# run new systems
for audio_file in CONFIG["audio_files"]:
    create_video()


### Run previous

In [ ]:
# run saved config
scenes_file_path = './images/DefundUnsound_20250221_191350/scenes.json'
audio_images_dir = './images/DefundUnsound_20250221_191350'
audio_videos_dir = './output/DefundUnsound_20250221_191350'
timestamp = '20250221_191350'
last_end_value = 232.5

with open(scenes_file_path, "r") as scenes_file:
    scenes = json.load(scenes_file)

process_audio_video(CONFIG, scenes, audio_images_dir, audio_videos_dir, last_end_value, timestamp)